<a href="https://colab.research.google.com/github/Ivan8Garcia/Proyecto_NoCountry/blob/main/Vuelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

datos=pd.read_csv("/content/flight_delays.csv")
datos.shape

(1747627, 16)

In [105]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1747627 entries, 0 to 1747626
Data columns (total 16 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   FlightID            int64 
 1   Airline             object
 2   FlightNumber        int64 
 3   Origin              object
 4   Destination         object
 5   ScheduledDeparture  object
 6   ActualDeparture     object
 7   ScheduledArrival    object
 8   ActualArrival       object
 9   DelayMinutes        int64 
 10  DelayReason         object
 11  Cancelled           bool  
 12  Diverted            bool  
 13  AircraftType        object
 14  TailNumber          object
 15  Distance            int64 
dtypes: bool(2), int64(4), object(10)
memory usage: 190.0+ MB


In [106]:
datos.isnull().sum()

,0
FlightID,0
Airline,0
FlightNumber,0
Origin,0
Destination,0
ScheduledDeparture,0
ActualDeparture,0
ScheduledArrival,0
ActualArrival,0
DelayMinutes,0


In [107]:
datos.fillna(0,inplace=True)

In [108]:
datos.isnull().sum()

,0
FlightID,0
Airline,0
FlightNumber,0
Origin,0
Destination,0
ScheduledDeparture,0
ActualDeparture,0
ScheduledArrival,0
ActualArrival,0
DelayMinutes,0


In [109]:
datos.head()

,FlightID,Airline,FlightNumber,Origin,Destination,ScheduledDeparture,ActualDeparture,ScheduledArrival,ActualArrival,DelayMinutes,DelayReason,Cancelled,Diverted,AircraftType,TailNumber,Distance
0,1,United,4558,ORD,MIA,2024-09-01 08:11,2024-09-01 08:30,2024-09-01 12:11,2024-09-01 12:19,8,Weather,True,False,Boeing 737,N71066,1031
1,2,Delta,8021,LAX,MIA,2024-09-01 10:25,2024-09-01 10:41,2024-09-01 13:25,2024-09-01 13:27,2,Air Traffic Control,True,True,Airbus A320,N22657,1006
2,3,Southwest,7520,DFW,SFO,2024-09-01 16:53,2024-09-01 17:05,2024-09-01 17:53,2024-09-01 18:07,14,Weather,True,True,Boeing 737,N95611,2980
3,4,Delta,2046,ORD,BOS,2024-09-01 14:44,2024-09-01 15:04,2024-09-01 18:44,2024-09-01 18:34,-10,0,False,False,Boeing 777,N90029,1408
4,5,Delta,6049,LAX,SEA,2024-09-01 01:51,2024-09-01 02:08,2024-09-01 05:51,2024-09-01 06:15,24,Air Traffic Control,False,True,Boeing 737,N27417,2298


##ELIMINACION DE COLUMNAS PARA MODELO MVP, SOLO NECESITAREMOS "Aerolínea","Origen", "Destino", "Fecha/Hora", "Retraso".

In [110]:
datos = datos[['Airline', 'Origin', 'Destination', 'ScheduledDeparture']]
datos = datos.rename(columns={'Airline': 'Aerolínea', 'Origin': 'Origen', 'Destination': 'Destino', 'ScheduledDeparture': 'Fecha/Hora'})
datos.head()

,Aerolínea,Origen,Destino,Fecha/Hora
0,United,ORD,MIA,2024-09-01 08:11
1,Delta,LAX,MIA,2024-09-01 10:25
2,Southwest,DFW,SFO,2024-09-01 16:53
3,Delta,ORD,BOS,2024-09-01 14:44
4,Delta,LAX,SEA,2024-09-01 01:51


In [111]:
datos['Fecha/Hora'] = pd.to_datetime(datos['Fecha/Hora'])
datos['Fecha'] = datos['Fecha/Hora'].dt.date
datos['Hora'] = datos['Fecha/Hora'].dt.time
datos.drop('Fecha/Hora', axis=1, inplace=True)
datos.head()

,Aerolínea,Origen,Destino,Fecha,Hora
0,United,ORD,MIA,2024-09-01,08:11:00
1,Delta,LAX,MIA,2024-09-01,10:25:00
2,Southwest,DFW,SFO,2024-09-01,16:53:00
3,Delta,ORD,BOS,2024-09-01,14:44:00
4,Delta,LAX,SEA,2024-09-01,01:51:00


In [116]:
import pandas as pd

# Re-reading the data to ensure DelayMinutes is present
# This assumes the original 'datos' DataFrame from flight_delays.csv
temp_datos = pd.read_csv("/content/flight_delays.csv")
temp_datos.fillna(0, inplace=True) # Apply previous NaN handling

# Select the required columns including DelayMinutes
datos = temp_datos[['Airline', 'Origin', 'Destination', 'ScheduledDeparture', 'DelayMinutes']].copy()

# Rename columns as requested by user in Spanish
datos = datos.rename(columns={
    'Airline': 'Aerolínea',
    'Origin': 'Origen',
    'Destination': 'Destino',
    'ScheduledDeparture': 'Fecha/Hora',
    'DelayMinutes': 'Retraso_minutos'
})

# Create the binary 'retrasado' column
datos['retrasado'] = (datos['Retraso_minutos'] > 15).astype(int)

# Split Fecha/Hora into Fecha and Hora
datos['Fecha/Hora'] = pd.to_datetime(datos['Fecha/Hora'])
datos['Fecha'] = datos['Fecha/Hora'].dt.date
datos['Hora'] = datos['Fecha/Hora'].dt.time
datos = datos.drop(columns=['Fecha/Hora'])

# Ensure the requested column order
datos = datos[['Aerolínea', 'Origen', 'Destino', 'Fecha', 'Hora', 'Retraso_minutos', 'retrasado']]

datos.head()

,Aerolínea,Origen,Destino,Fecha,Hora,Retraso_minutos,retrasado
0,United,ORD,MIA,2024-09-01,08:11:00,8,0
1,Delta,LAX,MIA,2024-09-01,10:25:00,2,0
2,Southwest,DFW,SFO,2024-09-01,16:53:00,14,0
3,Delta,ORD,BOS,2024-09-01,14:44:00,-10,0
4,Delta,LAX,SEA,2024-09-01,01:51:00,24,1


In [113]:
datos = datos[['Airline', 'Origin', 'Destination', 'ScheduledDeparture', 'DelayMinutes']]
datos['retrasado'] = (datos['DelayMinutes'] > 15).astype(int)
datos = datos.rename(columns={'Airline': 'Aerolínea', 'Origin': 'Origen', 'Destination': 'Destino', 'ScheduledDeparture': 'Fecha/Hora', 'DelayMinutes': 'Retraso_minutos'})
datos.head()

Entregables deseados

Notebook (Jupyter/Colab) del equipo de Data Science, que contenga:

Exploración y limpieza de datos (EDA);

Creación de variables relevantes (ej.: hora del vuelo, día de la semana, aeropuerto de origen/destino, aerolínea);

Entrenamiento de un modelo predictivo (ej.: Logistic Regression, Random Forest);

Evaluación del desempeño (Accuracy, Precision, Recall, F1-score);

Exportación del modelo serializado (joblib/pickle).